In [1]:
%load_ext autoreload
%autoreload 3

In [6]:
import pandas as pd
from pandasql import sqldf 
import data_helpers as dh
from identifiers import name_dict

In [33]:
game_info = dh.get_table('game_info')
all_picks = dh.get_table('all_picks')
all_picks_info = dh.get_table('all_picks_info')

## Comparisons

In [4]:
def get_weekly_comparison_table(week_num):
    df = dh.load_pickem_raw(week_num)
    first_row = df.index[df.isnull().all(1)].values[0]
    df = df[first_row+1:]
    df = df
    col_names = [name_dict[e] for e in df.iloc[0][1:8]]
    df = df.iloc[:, :9]
    df.columns = ['game_id'] + col_names + ['winner']
    df = df[2:]
    df = df[df['winner'].notnull()]
    return df.reset_index(drop=True)

In [7]:
weekly_comparison_lst = []
for i in range(1,19):
    weekly_comparison_lst.append(get_weekly_comparison_table(i))
comp_table = pd.concat(weekly_comparison_lst)

In [8]:
comp_table[comp_table['daniel'] != comp_table['winner']][['daniel', 'winner']].iloc[:50]

,daniel,winner
0,Chiefs,Lions
1,Panthers,Falcons
2,Texans,Ravens
3,Bengals,Browns
9,Bears,Packers
12,Patriots,Eagles
13,Seahawks,Rams
14,Giants,Cowboys
15,Bills,Jets
0,Vikings,PUSH


In [13]:
sum(comp_table['jordan'] == comp_table['daniel']) / len(comp_table)

0.5147058823529411

In [16]:
#for p1, p2 in all_combinations:
rows = []
for p1 in users:
    for p2 in users:
        table = comp_table[[p1, p2]]
        agree_rate = round(sum(comp_table[p1] == comp_table[p2]) / len(comp_table), 3)
        agree_table = comp_table[(comp_table[p1] == comp_table[p2]) & (comp_table['winner'] != 'PUSH')]
        disagree_table = comp_table[(comp_table[p1] != comp_table[p2]) & (comp_table['winner'] != 'PUSH')]
        win_pct_when_agree = round((sum(agree_table[p1] == agree_table['winner']) / len(agree_table)), 3)
        try:
            p1_win_pct_when_disagree = round((sum(disagree_table[p1] == disagree_table['winner']) / len(disagree_table)), 3)
        except ZeroDivisionError:
            p1_win_pct_when_disagree = None
        #print(p1, p2, agree_rate, win_pct_when_agree, p1_win_pct_when_disagree)
        rows.append([p1, p2, agree_rate, win_pct_when_agree, p1_win_pct_when_disagree])
columns = ['p1', 'p2', 'agree_rate', 'win_pct_when_agree', 'p1_win_pct_when_disagree']
comp_statistics = pd.DataFrame(rows, columns=columns)

In [17]:
comp_statistics.sort_values(by=['p1','agree_rate'])

,p1,p2,agree_rate,win_pct_when_agree,p1_win_pct_when_disagree
1,adam,andres,0.467,0.496,0.507
5,adam,james,0.467,0.549,0.460
3,adam,greg,0.493,0.528,0.477
4,adam,jordan,0.500,0.543,0.462
2,adam,daniel,0.515,0.496,0.508
6,adam,colin,0.551,0.518,0.483
0,adam,adam,0.993,0.506,0.000
13,andres,colin,0.434,0.504,0.479
9,andres,daniel,0.445,0.478,0.500
7,andres,adam,0.467,0.496,0.486


## ATS Bonus

In [39]:
def get_ats_bonus(week_num):
    df = dh.load_pickem_raw(week_num)
    first_row = df.index[df.isnull().all(1)].values[0]
    df = df[:first_row]
    df.drop(['Timestamp', 'Name', 'Email'], axis=1, inplace=True, errors='ignore')
    df.rename(columns={'Email Address': 'email', 'ATS Bonus':'bonus'}, inplace=True)
    df = df.loc[:, 'email':'bonus']
    df['week_num'] = week_num
    df['user_id'] = df['email'].apply(lambda x: name_dict[x])
    df['bonus'] = df['bonus'].str.strip()
    return df[['user_id','bonus','week_num']]

In [40]:
bonus_dfs = []
for i in range(1,19):
    bonus_dfs.append(get_ats_bonus(i))
bonus_df = pd.concat(bonus_dfs)

In [46]:
q = '''
select 
    user_id,
    bonus, 
    count(*) num_ats
from bonus_df
group by 1,2
order by 3 desc, 2
'''
df = sqldf(q)
df[df['user_id'] == 'greg']

,user_id,bonus,num_ats
11,greg,Broncos,2
12,greg,Browns,2
15,greg,Chiefs,2
24,greg,Raiders,2
28,greg,Texans,2
30,greg,Vikings,2
40,greg,Buccaneers,1
49,greg,Colts,1
56,greg,Eagles,1
66,greg,Patriots,1
